In [90]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import matplotlib.pyplot as plt
import csv
from cobra import Reaction


In [94]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_1cell.xml")
#Estimate inf
inf = 1e6



In [108]:
#This codeblock is to define some functions that will be used for modelling.


#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light
def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp


#This function is used to set the inputs to the model used. 
def define_model_medium(model, co2, o2, ppfd, 
                        medium_dir='../misc/photo_medium.csv', no3=inf, h2o=inf, h=inf, 
                        nh4=inf, pi=inf):
    model_photo_media = mm.read_medium_csv(medium_dir, model)
    model_photo_media['EX_no3(e)'] = no3
    model_photo_media['EX_h2o(e)'] = h2o
    model_photo_media['EX_h(e)'] = h
    model_photo_media['EX_nh4(e)'] = nh4
    model_photo_media['EX_co2(e)'] = co2
    model_photo_media['EX_o2(e)'] = o2
    model_photo_media['EX_photonVis(e)'] = ppfd
    model_photo_media['EX_pi(e)'] = pi
    #Set set model medium as model
#     print('Added model medium')
    return model_photo_media

    
def turn_off_cofac_cycles(model, inact_dir='../misc/leaf_inactivated.tsv'):
    file = csv.reader(open(inact_dir), delimiter='\t')
    leaf_inactive_rxns = list()
    for rows in file:
        for rxns in rows:
            leaf_inactive_rxns.append(rxns)
        
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            rxns.bounds = (0,0)
#     print('Successfully turned off cofactor-cycling reactions')

    
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")

def add_enzyme_constraints(model):
    #Maximum values for constraints
    wt_pepc = 0 #umol m-2 s-1 #Note: Need to constrain it to 0 pala since no PEPC was detected in either cell type
    wt_mdh = 11.18 #umol m-2 s-1
    wt_nadp_me = 0.14 #umol m-2 s-1
    wt_ppdk = 0.31 #umol m-2 s-1
    wt_CA = 7.5 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1 CA activity)

    #PEPC constraint (Reaction id: PPCc)
    #Need to constrain it to 0 since reaction is only detected in Vascular tissue
    pepc_M = model.reactions.PPCc
    wt_pepc_cons = model.problem.Constraint(pepc_M.flux_expression, 
                                            lb = 0, ub = wt_pepc)
    wt_pepc_cons.name = 'wt_pepc_cons'
    model.add_cons_vars(wt_pepc_cons)

    #PPDK constraints (Reaction id: PPDKs) (note that this is found in the chloroplast?) 
    #Not detected via immunolocalization but enzyme activity is detected

    ppdks_M = model.reactions.PPDKs
    wt_ppdks_cons = model.problem.Constraint(ppdks_M.flux_expression, 
                                             lb = 0, ub = wt_ppdk)
    wt_ppdks_cons.name = 'wt_ppdks_cons'
    model.add_cons_vars(wt_ppdks_cons)
    
    
    #Malate Dehydrogenase 
    #Only mitochondrial in WT Rice M cells
    mdhm_M = model.reactions.MDHm
    wt_mdh_cons = model.problem.Constraint(mdhm_M.flux_expression,
                                           lb= 0, ub=wt_mdh)
    wt_mdh_cons.name = "wt_mdh_cons"
    model.add_cons_vars(wt_mdh_cons)

    #NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
    #Let's see if I can force it to have a small amount of flux 
    nadp_me_M = model.reactions.MDHys

    wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression,
                                             lb= 0, ub=wt_nadp_me)
    wt_nadpme_cons.name = "wt_nadpme_cons"
    model.add_cons_vars(wt_nadpme_cons)


    #I should add constraints for Carbonic Anhydrase
    #I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
    #Flux is reversible so constraints are bi-directional
    hco3es_m = model.reactions.HCO3Es
    hco3ec_m = model.reactions.HCO3Ec
    hco3em_m = model.reactions.HCO3Em

    ca_cons = model.problem.Constraint(hco3es_m.flux_expression +
                                       hco3ec_m.flux_expression +
                                       hco3em_m.flux_expression,
                                      lb = -wt_CA, ub = wt_CA)
    ca_cons.name = 'Carbonic_anhydrase_constraint'
    model.add_cons_vars(ca_cons)
    #Rbcl constaints
    #Retrieve flux expressions oof each RBCl reaction
    
    
    rbpc_M = model.reactions.RBPCs.flux_expression
    rbpo_M = model.reactions.RBPOs.flux_expression

    #Constraint such that it is limited to 132 umol m-2 s-1
    rbcl_vcmax_cons = model.problem.Constraint(rbpc_M, lb = 0, ub= 132)
    rbcl_vcmax_cons.name='rbcl_vcmax_cons'
    model.add_cons_vars(rbcl_vcmax_cons)
    #Constraints for rbcl flux such that v_c/v_o = 3 or higher.
    rbcl_vcvo = model.problem.Constraint(3*(rbpo_M) 
                                         - 1*(rbpc_M),
                                         lb=0,ub=1000)
    rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
    model.add_cons_vars(rbcl_vcvo)

    #Turn off the RBPC2s reactions since we already defined the constraints above
    model.reactions.RBPC2s.bounds = (0,0)
    
    
    
    #What if I simply constrained that of the M cell one to 3:1?
    #This constraint is pretty good actually. 
    #This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
    #wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.
# #This code block is to set a constraint such that M-to-BS cell NGAM ratio is 10-to-1 
# #Similar to what Moreno-Villena (2021) (preprint) had done 

#This function takes two arguments: the model and the maximal  ppfd input to the system
def add_ngam_cons(model, ppfd):
    #Retrieve NGAM reactions
    ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c')
    ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s')
    ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m')
    ngam_atp_c_M = mm.get_rxn(model, 'ngam_atp_c')
    #Set Fixed fluxes
    nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
    nadphox_c_s_M.name = "nadphox_cs_ratio_M"
    nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
    nadphox_s_m_M.name = "nadphox_sm_ratio_M"


    #Add constraints
    model.add_cons_vars(nadphox_c_s_M)
    model.add_cons_vars(nadphox_s_m_M)

    #Retrieve flux expressionns
    fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
    fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
    fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)
    fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_c_M)

    #Set the constraint between ATP:NADPH NGAM to 3:1
    nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M) 
                                         - 1*(fex_atp_c_M),
                                         lb=0,ub=0)
    nadphox_atpase.name = "nadphox_atpase_ratio"
    model.add_cons_vars(nadphox_atpase)
    #Compute NGAM value and add constraint as a lower bound/upper bound to model
    ngam_value = compute_ngam_atp(ppfd)
    ngam_cons = model.problem.Constraint(fex_atp_c_M, lb=ngam_value, ub=ngam_value)
    ngam_cons.name = 'NGAM_ATP_constraint'
    model.add_cons_vars(ngam_cons)
    
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above

def print_summary(sample_fluxes_df):
    print('summary of fluxes')
    print('rbcl M cell: ', sample_fluxes['RBPCs'])
    print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs'])
    print('vc/vo M:', sample_fluxes['RBPCs']/sample_fluxes['RBPOs'])
    print('PEPC M', sample_fluxes['PPCc'])
    print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec'])
    print('NADP-ME M', sample_fluxes['MDHys'])
    print('Biomass M: ', sample_fluxes['Straw_Biomass'])
    print('Phloem M: ', sample_fluxes['DM_Phloem'])
    print('co2 consumption M', sample_fluxes['CO2tex'])
    print('o2 consumption M', sample_fluxes['O2tex'])
    print('Photosystem II M', sample_fluxes['PSIINC'])
    print('PSI M', sample_fluxes['PSIMR'])
    print('PPFD M: ', sample_fluxes['PRISM_white_LED'])
    print('ATP synthesis (stromal) M', sample_fluxes['ATPSs'], 'ATP synthase (mit) M', sample_fluxes['ATPSm'])
    pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
    pd_abs_flux = 0
    for pds in pd_rxn:
        pd_abs_flux += abs(sample_fluxes[pds.id])
    
    print('pd_abs_flux: ', pd_abs_flux)

In [ ]:
    
    file = csv.reader(open(inact_dir), delimiter='\t')
    leaf_inactive_rxns = list()
    for rows in file:
        for rxns in rows:
            leaf_inactive_rxns.append(rxns)
        
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            rxns.bounds = (0,0)

In [109]:
# Let's try restricting O2 intake again to a specific amount, let's see if PPFD responds accordingly

with model as m1:
    
#Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to reflect Mature leaf
    m1.medium = define_model_medium(m1, co2=inf, o2=2.2618, ppfd=1000, h=0, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
    add_enzyme_constraints(m1)
    #Add trans constraints
    add_ngam_cons(m1, 1000)
    
    #Change objective 
    #Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
    #Remove thermodynamically infeasible loops
    sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)
    
    sample_fluxes_df = sample_fluxes.to_frame()

print('\n')
print_summary(sample_fluxes_df)




summary of fluxes
rbcl M cell:  101.69478046657191
rbcl M cell (photorespiration) 33.89826015552397
vc/vo M: 3.0
PEPC M 0.0
Carbonic Anhydrase (Cytosolic) M -0.13055844530896252
NADP-ME M 0.01320575506308952
Biomass M:  0.0
Phloem M:  0.0
co2 consumption M 84.62866566077898
o2 consumption M 2.2618
Photosystem II M 120.85164130398918
PSI M 120.85164130398918
PPFD M:  1000.0
ATP synthesis (stromal) M 161.13552173865224 ATP synthase (mit) M 0.0
pd_abs_flux:  0


In [99]:

ppfd_solns_2cell = pd.DataFrame(index=list(i.id for i in model.reactions))

#Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)
for ppfd in range(0,2050,50):
    with model as m1:

        #Set medium to change photon flux and re-add to model
       #Change objective function to reflect Mature leaf
        m1.medium = define_model_medium(m1, co2=26, o2=2.2618, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
        turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
        add_enzyme_constraints(m1)
        #Add trans constraints
        add_ngam_cons(m1, ppfd)
        model.reactions.FDHNc.bounds = (0, 40)
        #Change objective 
        #Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
        #Change objective function to mature leaf
        model.reactions.get_by_id('Straw_Biomass').objective_coefficient = 0
        mm.get_rxn(model,'DM_Phloem').objective_coefficient = 1
        sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
        #Remove thermodynamically infeasible loops
        sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)

        sample_fluxes_df = sample_fluxes.to_frame()

        #perform pFBA & convert to dataframe
        tc_soln = cobra.flux_analysis.pfba(model).to_frame()
        tc_soln = pd.DataFrame({ppfd:tc_soln['fluxes']}, index=list(tc_soln.index))
        print("PPFD:", ppfd, " RBPC_M cell flux:", tc_soln[ppfd]['RBPCs'])
        #Append to Dataframe
        ppfd_solns_2cell[ppfd] = tc_soln

#Code now works

PPFD: 0  RBPC_M cell flux: 2.499274037115341
PPFD: 50  RBPC_M cell flux: 7.459049358588173
PPFD: 100  RBPC_M cell flux: 12.418824680060965
PPFD: 150  RBPC_M cell flux: 17.37860000153381
PPFD: 200  RBPC_M cell flux: 22.33837532300656
PPFD: 250  RBPC_M cell flux: 27.29815064447941
PPFD: 300  RBPC_M cell flux: 31.935413602160303
PPFD: 350  RBPC_M cell flux: 35.327899637285995
PPFD: 400  RBPC_M cell flux: 38.738696048955504
PPFD: 450  RBPC_M cell flux: 42.14949246062625
PPFD: 500  RBPC_M cell flux: 45.560288872298436
PPFD: 550  RBPC_M cell flux: 48.50020411062225
PPFD: 600  RBPC_M cell flux: 51.25075465942945
PPFD: 650  RBPC_M cell flux: 54.00130520823744
PPFD: 700  RBPC_M cell flux: 56.75185575704606
PPFD: 750  RBPC_M cell flux: 59.50240630585422
PPFD: 800  RBPC_M cell flux: 62.252956854661214
PPFD: 850  RBPC_M cell flux: 65.00350740347122
PPFD: 900  RBPC_M cell flux: 67.75405795227817
PPFD: 950  RBPC_M cell flux: 70.5046085010863
PPFD: 1000  RBPC_M cell flux: 73.25515904989432
PPFD: 1050

In [ ]:
model.medium

In [ ]:
model.reactions.RBPCs_BS

In [ ]:
for rxns in model.reactions:
    print(rxns.id, sample_fluxes[rxns.id])

In [ ]:
#Note:
#Unconstrained CO2 intake leads to a saturated  net CO2 assimilation rate of around 40 umol m-2 s-1
#Better to constrain this to known uptake rates instead


In [ ]:
#Note:
#If I restrict the flux of H2O to the BS cell only what happens is that CO2 is transported higher to the M cell rather than 

In [ ]:
#Oh wtf it works, below 500 ppfd no net intake below dark respiration is observed tapos nasasaturate siya agad to  the max value
#Cooooool. Based on Weber et al (1986) the critical point is at around 500-600 umol he m-2 s-1
#Weber, J. A., Tenhunen, J. D., Gates, D. M., & Lange, O. L. (1987). Effect of photosynthetic photon flux density on carboxylation efficiency. Plant physiology, 85(1), 109-114.


In [ ]:
#What if I limited influx of CO2 from the M cell to the BS cell?

In [ ]:
#What I've done was unconstrained CO2 exchange to allow for CO2 efflux. At around 300-400 ppfd a net release of CO2 is observed from the mdoel, consistent 
#with Nobel (2020)'s  'Leaves and Fluxes'
#I need to read more on this. This is actually quite interesting
# 

In [ ]:
#Constraining O2 consumption to around 3.312 forces water to be consumed by the system, unlike when O2 is unconstrained.
#However, the same behavior when it comes to CO2 assimilation still is being produced.
#Constraining O2 consumption also forces Nitrate consumption to be constrained to realistic levels (when compared to non-constrained O2 where consumption was 10-fold )
#I suspect that this is definitely due to how PPFD flux is structured. 
#A cop-out to this would be to double PPFD per run and measuring PPFD flux in the specific reactions instead.

In [ ]:
#I can probably do the benchmarking right now. The model behaves quite interestingly actually

In [ ]:
#Note: Turning off other prism reactions (red, green, blue, etc) while retaining only white light reactions lowers net carbon assimilation
#I think this is in consideration on how white light is partitioned to several wavelengths, which prevents it from being fully utilized.

In [ ]:
#Other artifacts as of the moment include the following:
#model doesn't produce any Oxygen. Maybe I should add a specific demand reaction for it?
#Based on FVA, the model now produces some oxygen via the demand reaction. 

In [ ]:
#Model also facilitates transfer of metabolites unlike before.

In [ ]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.